# Step 6B · Sequence Models

Create 7-day behavioral sequences to train LSTM / GRU / CNN models that predict
weekly burnout level.

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import matplotlib.pyplot as plt

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DATA_DIR = Path('../data/processed')
MODEL_DIR = Path('../models/saved')
MODEL_DIR.mkdir(parents=True, exist_ok=True)

In [ ]:
daily_path = DATA_DIR / 'daily_with_burnout.parquet'
daily = pd.read_parquet(daily_path)
daily['date'] = pd.to_datetime(daily['date'])
daily = daily.sort_values(['user_id', 'date'])
feature_cols = [
    'sleep_hours', 'sleep_quality', 'work_hours', 'meetings_count',
    'tasks_completed', 'exercise_minutes', 'steps_count', 'caffeine_mg',
    'alcohol_units', 'screen_time_hours', 'stress_level', 'mood_score',
    'energy_level', 'focus_score', 'work_pressure'
]
window = 7

In [ ]:
def build_sequences(df, features, window):
    sequences, labels = [], []
    for uid, group in df.groupby('user_id'):
        feats = group[features].to_numpy(dtype=np.float32)
        labs = group['burnout_level'].to_numpy(dtype=np.int64)
        if len(group) < window:
            continue
        for idx in range(window, len(group) + 1):
            seq = feats[idx - window: idx]
            label = labs[idx - 1]
            if np.isnan(seq).any():
                continue
            sequences.append(seq)
            labels.append(label)
    return np.stack(sequences), np.array(labels)

seq_X, seq_y = build_sequences(daily, feature_cols, window)
seq_X.shape, seq_y.shape

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(seq_X, seq_y, test_size=0.2, stratify=seq_y, random_state=42)
train_ds = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))
val_ds = TensorDataset(torch.from_numpy(X_val), torch.from_numpy(y_val))
train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=256, shuffle=False)
seq_len = X_train.shape[1]
input_dim = X_train.shape[2]
num_classes = len(np.unique(seq_y))

In [ ]:
class SequenceNet(nn.Module):
    def __init__(self, input_dim, hidden_dim=128, cell='lstm'):
        super().__init__()
        rnn_cls = nn.LSTM if cell == 'lstm' else nn.GRU
        self.rnn = rnn_cls(input_dim, hidden_dim, batch_first=True, num_layers=2, dropout=0.2)
        self.head = nn.Sequential(
            nn.LayerNorm(hidden_dim)
            , nn.ReLU()
            , nn.Dropout(0.3)
            , nn.Linear(hidden_dim, num_classes)
        )

    def forward(self, x):
        out, _ = self.rnn(x)
        last = out[:, -1, :]
        return self.head(last)

class CNN1D(nn.Module):
    def __init__(self, input_dim, seq_len):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv1d(input_dim, 64, kernel_size=3, padding=1)
            , nn.ReLU()
            , nn.BatchNorm1d(64)
            , nn.Conv1d(64, 128, kernel_size=3, padding=1)
            , nn.ReLU()
            , nn.AdaptiveAvgPool1d(1)
        )
        self.fc = nn.Sequential(
            nn.Flatten()
            , nn.Dropout(0.3)
            , nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = x.transpose(1, 2)
        feats = self.conv(x)
        return self.fc(feats)

In [ ]:
def train_model(model, train_loader, val_loader, epochs=40, lr=1e-3, name='model'):
    model = model.to(DEVICE)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    best_val = float('inf')
    history = {'train': [], 'val': []}
    for epoch in range(1, epochs + 1):
        model.train()
        tr_losses = []
        for xb, yb in train_loader:
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)
            optimizer.zero_grad()
            loss = criterion(model(xb), yb)
            loss.backward()
            optimizer.step()
            tr_losses.append(loss.item())
        model.eval()
        val_losses = []
        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(DEVICE), yb.to(DEVICE)
                val_losses.append(criterion(model(xb), yb).item())
        train_loss = np.mean(tr_losses)
        val_loss = np.mean(val_losses)
        history['train'].append(train_loss)
        history['val'].append(val_loss)
        if val_loss < best_val:
            best_val = val_loss
            torch.save({'state_dict': model.state_dict(), 'feature_cols': feature_cols}, MODEL_DIR / f'{name}.pt')
        if epoch % 5 == 0:
            print(f
    return history

In [ ]:
hist_lstm = train_model(SequenceNet(input_dim, cell='lstm'), train_loader, val_loader, name='lstm_classifier')
hist_gru = train_model(SequenceNet(input_dim, cell='gru'), train_loader, val_loader, name='gru_classifier')
hist_cnn = train_model(CNN1D(input_dim, seq_len), train_loader, val_loader, name='cnn1d_classifier')

In [ ]:
def plot_history(history, title):
    plt.figure(figsize=(7, 4))
    plt.plot(history['train'], label='train')
    plt.plot(history['val'], label='val')
    plt.title(title)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.tight_layout()

plot_history(hist_lstm, 'LSTM Loss')
plot_history(hist_gru, 'GRU Loss')
plot_history(hist_cnn, 'CNN Loss')

In [ ]:
def evaluate_model(model_path):
    payload = torch.load(model_path, map_location=DEVICE)
    state = payload['state_dict']
    name = model_path.stem
    if 'lstm' in name:
        model = SequenceNet(input_dim, cell='lstm')
    elif 'gru' in name:
        model = SequenceNet(input_dim, cell='gru')
    else:
        model = CNN1D(input_dim, seq_len)
    model.load_state_dict(state)
    model = model.to(DEVICE)
    model.eval()
    with torch.no_grad():
        xb = torch.from_numpy(X_val).to(DEVICE)
        logits = model(xb)
        preds = torch.argmax(logits, dim=1).cpu().numpy()
    return preds

for model_name in ['lstm_classifier.pt', 'gru_classifier.pt', 'cnn1d_classifier.pt']:
    preds = evaluate_model(MODEL_DIR / model_name)
    acc = accuracy_score(y_val, preds)
    f1 = f1_score(y_val, preds, average='macro')
    print(model_name, {'accuracy': acc, 'f1_macro': f1})